# 车道线识别

# 版本：5
# 带霍夫变换的车道线检测：梯形区域+Sobel阈值分割

# 库函数

In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 带区域选择的阈值分割

In [3]:
# 单张图像处理：为了后面好改
img = cv2.imread('../pics/img5_16.jpg', 1)              # 这里是彩色图像

[ysize, xsize, channel] = img.shape             # (368, 640, 3)
res = np.copy(img)                              # 以防后面要叠加，先复制
imgGray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)  # 转为灰度图
thresh = 10                                     # 要求Sobel梯度大于的值

# 区域顶点
# 应该区域往下点，就可以让max_line_gap大一点，好连成直线
left_bottom = [236, 368]
right_bottom = [351, 368]
left_top = [296, 240]
right_top = [315,240]

# 区域边界直线
fit_left = np.polyfit((left_bottom[0], left_top[0]), (left_bottom[1], left_top[1]), 1)
fit_right = np.polyfit((right_bottom[0], right_top[0]), (right_bottom[1], right_top[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)
fit_top = np.polyfit((left_top[0], right_top[0]), (left_top[1], right_top[1]), 1)

# 梯形区域Mask：region_thresholds
XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
region_thresholds = (YY > (XX * fit_left[0] + fit_left[1])) & \
                    (YY > (XX * fit_right[0] + fit_right[1])) & \
                    (YY < (XX * fit_bottom[0] + fit_bottom[1])) & \
                    (YY > (XX * fit_top[0] + fit_top[1]))

# Sobel算子
cur = cv2.GaussianBlur(imgGray, (3, 3), 1)
imgSobel = cv2.Canny(cur, 50, 200)
imgSobel[~region_thresholds] = 0
cv2.imshow("imgSobel", imgSobel)                  # 查看
cv2.waitKey(0)

# Hough变换灰度图：阈值分割+区域选择后的imgSobel
rho = 1
theta = np.pi/180
threshold = 1
min_line_length = 10
max_line_gap = 10

lines = cv2.HoughLinesP(imgSobel, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)

# 绘制直线：彩色图res
if lines is not None:
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(res,(x1, y1), (x2, y2), (0, 255, 0), 2)
else :
    print("None Lines!")
    
cv2.imshow("image", res)                  # 查看
cv2.waitKey(0)

None Lines!


-1

In [14]:
# 图片：每个视频37张
file_path = '../pics/'
save_path = '../save5/'                    # 存储到save5文件夹
for i in range(37):
    imgName = file_path + "img1_" + str(i)
    imgPath = imgName + ".jpg"
    
    img = cv2.imread(imgPath, 1)                    # 这里是RGB图像

    [ysize, xsize, channel] = img.shape             # (368, 640, 3)
    res = np.copy(img)                              # 以防后面要叠加，先复制
    imgGray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)  # 转为灰度图
    thresh = 10                                     # 要求Sobel梯度大于的值

    # 区域顶点
    # 应该区域往下点，就可以让max_line_gap大一点，好连成直线
    left_bottom = [236, 368]
    right_bottom = [351, 368]
    left_top = [296, 240]
    right_top = [315,240]

    # 区域边界直线
    fit_left = np.polyfit((left_bottom[0], left_top[0]), (left_bottom[1], left_top[1]), 1)
    fit_right = np.polyfit((right_bottom[0], right_top[0]), (right_bottom[1], right_top[1]), 1)
    fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)
    fit_top = np.polyfit((left_top[0], right_top[0]), (left_top[1], right_top[1]), 1)

    # 梯形区域Mask：region_thresholds
    XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
    region_thresholds = (YY > (XX * fit_left[0] + fit_left[1])) & \
                        (YY > (XX * fit_right[0] + fit_right[1])) & \
                        (YY < (XX * fit_bottom[0] + fit_bottom[1])) & \
                        (YY > (XX * fit_top[0] + fit_top[1]))

    # Sobel算子
    cur = cv2.GaussianBlur(imgGray, (3, 3), 1)
    imgSobel = cv2.Canny(cur, 50, 200)
    imgSobel[~region_thresholds] = 0
    
    cv2.imshow("imgSobel", imgSobel)                  # 查看
    cv2.waitKey(0)

    # Hough变换灰度图：阈值分割+区域选择后的imgSobel
    rho = 1
    theta = np.pi/180
    threshold = 1
    min_line_length = 10
    max_line_gap = 10

    lines = cv2.HoughLinesP(imgSobel, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # 绘制直线：彩色图res
    if lines is not None:                   # 可能没有直线，就不处理
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(res,(x1, y1), (x2, y2), (0, 255, 0), 2)
    else :
        print("None Lines!")
        
    cv2.imshow("res", res)                  # 查看
    cv2.waitKey(0)
    
    cv2.imwrite(save_path + "img1_" + str(i) + ".jpg", res)


In [4]:
# 晚上图片：每个视频37张
file_path = '../pics/'
save_path = '../save5/'           # 存储到save5文件夹
for i in range(37):
    imgName = file_path + "img5_" + str(i)
    imgPath = imgName + ".jpg"
    
    img = cv2.imread(imgPath, 1)                    # 这里是RGB图像
    
    [ysize, xsize, channel] = img.shape             # (368, 640, 3)
    res = np.copy(img)                              # 以防后面要叠加，先复制
    imgGray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)  # 转为灰度图
    thresh = 10                                     # 要求Sobel梯度大于的值

    # 区域顶点
    # 应该区域往下点，就可以让max_line_gap大一点，好连成直线
    left_bottom = [236, 368]
    right_bottom = [351, 368]
    left_top = [296, 240]
    right_top = [315,240]

    # 区域边界直线
    fit_left = np.polyfit((left_bottom[0], left_top[0]), (left_bottom[1], left_top[1]), 1)
    fit_right = np.polyfit((right_bottom[0], right_top[0]), (right_bottom[1], right_top[1]), 1)
    fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)
    fit_top = np.polyfit((left_top[0], right_top[0]), (left_top[1], right_top[1]), 1)

    # 梯形区域Mask：region_thresholds
    XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
    region_thresholds = (YY > (XX * fit_left[0] + fit_left[1])) & \
                        (YY > (XX * fit_right[0] + fit_right[1])) & \
                        (YY < (XX * fit_bottom[0] + fit_bottom[1])) & \
                        (YY > (XX * fit_top[0] + fit_top[1]))

    # Sobel算子
    cur = cv2.GaussianBlur(imgGray, (3, 3), 1)
    imgSobel = cv2.Canny(cur, 50, 200)
    imgSobel[~region_thresholds] = 0
    
    # cv2.imshow("imgSobel", imgSobel)                  # 查看
    # cv2.waitKey(0)

    # Hough变换灰度图：阈值分割+区域选择后的imgSobel
    rho = 1
    theta = np.pi/180
    threshold = 1
    min_line_length = 10
    max_line_gap = 10

    lines = cv2.HoughLinesP(imgSobel, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # 绘制直线：彩色图res
    if lines is not None:                   # 可能没有直线，就不处理
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(res,(x1, y1), (x2, y2), (0, 255, 0), 2)
    else :
        print("None Lines!")
    
    cv2.imshow("res", res)                  # 查看
    cv2.waitKey(0)
    
    cv2.imwrite(save_path + "img5_" + str(i) + ".jpg", res)

None Lines!
None Lines!
None Lines!
